In [1]:
%load_ext autoreload
%autoreload 2

from mt.scan import *
cle.select_device("RTX")

(OpenCL) NVIDIA GeForce RTX 4070 SUPER (OpenCL 3.0 CUDA)
	Vendor:                      NVIDIA Corporation
	Driver Version:              560.70
	Device Type:                 GPU
	Compute Units:               56
	Global Memory Size:          12281 MB
	Maximum Object Size:         3070 MB
	Max Clock Frequency:         2505 MHz
	Image Support:               Yes

In [2]:
path = "../../04_uCT/AD67/"
scan = Scan(path,
            discard_ends=True)
scan.load(logging=True,
          refresh=True)

Loading images from:  ../../04_uCT/AD67/Slices/
Loaded stack with shape (1781, 126, 2231) and a size of 1.00 GB in 1.22 s.
Loaded mask from: ../../04_uCT/AD67/segmentation.npy
Loaded particle mask from: ../../04_uCT/AD67/particle_mask.npy


In [3]:
# scan.try_segmentation_settings()

In [4]:
# scan.try_segmentation_settings(segment_particles_only=True)


In [5]:
settings = SegmentationSettings(particle_mask_sigma=0.4,
                                particle_n_erosions=3,
                                particle_enlarge_radius=1)
scan.set_particle_segmentation_settings(settings)
print(scan.particle_segmentation_settings)

air_mask_simga = 0.6
air_n_erosions = 2
particle_mask_sigma = 0.4
particle_n_erosions = 3
particle_enlarge_radius = 1
smooth_labels_radius = 2
contrast_min_percentile = 0
contrast_max_percentile = 100


In [6]:
settings = SegmentationSettings(air_mask_sigma=0.6,
                                air_n_erosions=2,
                                contrast_min_percentile=0,
                                contrast_max_percentile=9,
                                particle_mask_sigma=0.2,
                                particle_n_erosions=2,
                                particle_enlarge_radius=1,
                                smooth_labels_radius=2)
scan.set_segmentation_settings(settings)
print(scan.segmentation_settings)

air_mask_simga = 0.6
air_n_erosions = 2
particle_mask_sigma = 0.2
particle_n_erosions = 2
particle_enlarge_radius = 1
smooth_labels_radius = 2
contrast_min_percentile = 0
contrast_max_percentile = 9


In [7]:
scan.segment()

In [8]:
scan.segment_particles()

In [9]:
scan.show()

In [10]:
scan.save()